In [40]:
#! pip install ultralytics 
#! pip install psycopg2
import numpy as np
#import cv2
#import matplotlib.pyplot as plt
import os
import psycopg2
#import im_utils
import db_ops
import pandas as pd

#### Get the list of test labels

In [41]:
test_lbls = ['5-hour energy', '99', 'AW', 'Acadia', 'Arizona', 'Bang', 'Big_Gulp', 'Bill_Miller', 'Boulevard_Brewing', 'Brisk', 'Bud_light', 'Budweiser', 'Burger_King', 'Busch', 'Camel', 'Cheetos', 'Coca_Cola', 'Coors', 'Corona', 'Dasani', 'Dasy', 'David', 'Deer_Park', 'Dorritos', 'Dunkin_Donuts', 'Evan_Williams', 'Fanta', 'Fireball', 'Flow', 'Fruit_Punch', 'Gatorade', 'GoGo', 'Google', 'Goya', 'Grace', 'Green_Mountain', 'Heineken', 'Hersheys', 'Hill Country Fair', 'Icehouse_Edge', 'Jarritos', 'Jim_Beam', 'Kirkland', 'KitKat', 'KoolAid', 'La_Yogurt', 'LaCroix', 'Lays', 'Lybbis', 'Marlboro', 'McDonalds', 'Mentos', 'Michelob', 'Miller', 'Milwakee_Best_Ice', 'MnM', 'Modelo', 'Monster', 'Mountain_Dew', 'Natural_Ice', 'New_Amsterdam', 'Niagara', 'Oreo', 'pbr', 'Pacifico', 'Pampers', 'Parliament', 'Peaceful_Fruits', 'Perrier', 'Poland_Spring', 'Popeyes', 'PurAqua', 'Redbull', 'Ross Ouzo', 'Sams_Club', 'Samuel_Adams', 'Sangria_Senorial', 'Sanpple', 'Seven_eleven', 'Shasta', 'Silver_Falls', 'Slurpee', 'Smirnoff', 'Snickers', 'Sprite', 'Stella', 'Turkey_Hill', 'Twix', 'Two Ball', 'Vitamin', 'Welchs', 'Yoohoo']
test_lbls = [x.lower() for x in test_lbls]
test_lbls

['5-hour energy',
 '99',
 'aw',
 'acadia',
 'arizona',
 'bang',
 'big_gulp',
 'bill_miller',
 'boulevard_brewing',
 'brisk',
 'bud_light',
 'budweiser',
 'burger_king',
 'busch',
 'camel',
 'cheetos',
 'coca_cola',
 'coors',
 'corona',
 'dasani',
 'dasy',
 'david',
 'deer_park',
 'dorritos',
 'dunkin_donuts',
 'evan_williams',
 'fanta',
 'fireball',
 'flow',
 'fruit_punch',
 'gatorade',
 'gogo',
 'google',
 'goya',
 'grace',
 'green_mountain',
 'heineken',
 'hersheys',
 'hill country fair',
 'icehouse_edge',
 'jarritos',
 'jim_beam',
 'kirkland',
 'kitkat',
 'koolaid',
 'la_yogurt',
 'lacroix',
 'lays',
 'lybbis',
 'marlboro',
 'mcdonalds',
 'mentos',
 'michelob',
 'miller',
 'milwakee_best_ice',
 'mnm',
 'modelo',
 'monster',
 'mountain_dew',
 'natural_ice',
 'new_amsterdam',
 'niagara',
 'oreo',
 'pbr',
 'pacifico',
 'pampers',
 'parliament',
 'peaceful_fruits',
 'perrier',
 'poland_spring',
 'popeyes',
 'puraqua',
 'redbull',
 'ross ouzo',
 'sams_club',
 'samuel_adams',
 'sangria_se

In [42]:
label_path = './test_dataset/train/labels'
files = os.listdir(label_path)
file_names = []
labels = []
for file in files:
    file_labels = []
    with open(label_path + '/'+file, 'r') as label_file:
        for line in label_file:
            idx = int(line.split(' ')[0])
            file_labels.append(test_lbls[idx])
        labels.append(','.join(file_labels))
        file_names.append(file.split('_jpg')[0])
        
test_labels_df = pd.DataFrame({'file_name':file_names,'true': labels})
test_labels_df = test_labels_df.set_index('file_name')
test_labels_df['num_true'] = test_labels_df['true'].apply(lambda x: len(x.split(',')) if x else 1)
#test_labels_df.fillna({'num_true':1}, inplace = True)
test_labels_df

,true,num_true
file_name,,
IMG_1189,kirkland,1
xoZyrHIdNGg7n7yvPoJfKrqlcHj0Dw5BFT67Uq87,,1
IMG_3150,,1
IMG_2834,"bud_light,bud_light",2
IMG_4260,kirkland,1
...,...,...
IMG_3016,,1
tTzh0I2nNTMsC5QXKatN6B9jhqO3oiJfiIeGHZO4,david,1
gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,"corona,corona,corona,corona,corona,corona,coro...",21


#### Get the full list of avavliable images

In [43]:
query = f'''
SELECT bx.box_id as box_id
	,im.s3_uri AS uri
	, br.brand_name as brand
FROM box AS bx 
	LEFT JOIN image AS im
		ON im.image_id = bx.image_id
	LEFT JOIN brand as br
		ON br.brand_id = bx.brand_id 
ORDER BY bx.box_id	;
'''
boxes_df = db_ops.select_to_pandas(query, True, True)

/home/art_usr/source_code/litter-imagery-brand-recognition/db_ops.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [44]:
boxes_df['file_name'] = boxes_df['uri'].apply(lambda x: x.split('/')[-1].split('.')[0])
processed_images = boxes_df.groupby('file_name')['brand'].apply(list).reset_index(name='predicted')
processed_images = processed_images.set_index('file_name')
processed_images['num_found'] = processed_images['predicted'].apply(lambda x: len(x))
processed_images['predicted'] = processed_images['predicted'].apply(lambda x: str(x)[1:-1])
processed_images['predicted'] = processed_images['predicted'].apply(lambda x: x.replace("'", ''))
processed_images['predicted'] = processed_images['predicted'].apply(lambda x: x.replace(" ", ''))
#processed_images.sort_values('num_found').head(50)

In [45]:
result = test_labels_df.join(processed_images)
result['true'] = result['true'].apply(lambda x: 'nan' if x == '' else x)
result.fillna({'num_found':1}, inplace = True)
result.fillna({'predicted':'nan'}, inplace = True)

result.head(3)

,true,num_true,predicted,num_found
file_name,,,,
IMG_1189,kirkland,1,kirkland,1.0
xoZyrHIdNGg7n7yvPoJfKrqlcHj0Dw5BFT67Uq87,nan,1,"zz_unbranded,zz_unbranded",2.0
IMG_3150,nan,1,busch_light,1.0


In [46]:
def compare(true_str:str, predicted:str):
    true_brands = true_str.split(',')
    predicted_brands = predicted.split(',')
    correct_lst = []
    missed_lst = []
    for brand in true_brands:
        if brand in predicted_brands:
            correct_lst.append(brand)
            predicted_brands.remove(brand)
        else:
            missed_lst.append(brand)
    correct = len(correct_lst)
    missed = len(missed_lst)
    extra = len(predicted_brands)
    if predicted == 'nan':
        extra = 0
    if true_str == 'nan':
        missed = 0
    return correct, missed, extra

result['correct']=0
result['missed']=0
result['extra']=0
result['idx'] = range(len(result))
result.set_index('idx', inplace= True)

for i in result.index:
    correct, missed, extra = compare(result['true'].iloc[i], result['predicted'].iloc[i])
    result.loc[i, 'correct'] = correct
    result.loc[i, 'missed'] = missed
    result.loc[i, 'extra'] = extra
result.to_csv('resuts.csv', sep = '\t')
result

,true,num_true,predicted,num_found,correct,missed,extra
idx,,,,,,,
0,kirkland,1,kirkland,1.0,1,0,0
1,nan,1,"zz_unbranded,zz_unbranded",2.0,0,0,2
2,nan,1,busch_light,1.0,0,0,1
3,"bud_light,bud_light",2,zz_unbranded,1.0,0,2,1
4,kirkland,1,nan,1.0,0,1,0
...,...,...,...,...,...,...,...
291,nan,1,corona,1.0,0,0,1
292,david,1,zz_unbranded,1.0,0,1,1
293,"corona,corona,corona,corona,corona,corona,coro...",21,"heineken,heineken,corona",3.0,3,18,0


In [47]:
result['num_found'].sum()

424.0

In [48]:
result['num_true'].sum()

440

In [49]:
result['correct'].sum()

174

In [50]:
result['missed'].sum()

210

In [52]:
rec = 100*result['correct'].sum()/result['num_true'].sum()
prec = 100*result['correct'].sum()/result['num_found'].sum()
print(f'Precision: {round(prec,1)}, Recall: {round(rec,1)}')

Precision: 41.0, Recall: 39.5
